In [10]:
import sys
import numpy as np
import random
import pickle
import os

import torch
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms
from torchvision.datasets import CIFAR10

import models

sys.path.append("../../../FedLab/")

from fedlab.utils.dataset import functional as dataF
from fedlab.utils import SerializationTool

In [6]:
x = {'x':2, 'y': 1}
for key in sorted(x.keys()):
    print(key)

x
y


42.66666666666667

In [11]:
trainset = CIFAR10(root="../../../data/CIFAR10/", train=True, 
                   download=True, 
                   transform=transforms.ToTensor())

Files already downloaded and verified


In [12]:
train_loader = DataLoader(trainset,
            batch_size=16)

In [13]:
for imgs, targets in train_loader:
    tmp1 = targets.reshape(-1).long()
    tmp2 = list(targets.size())[0]
    break

In [17]:
targets.shape[0]

16

In [19]:
def get_mdl_params(model_list, n_par=None):
    if n_par is None:
        exp_mdl = model_list[0]
        n_par = 0
        for name, param in exp_mdl.named_parameters():
            n_par += len(param.data.reshape(-1))

    param_mat = np.zeros((len(model_list), n_par)).astype('float32')
    for i, mdl in enumerate(model_list):
        idx = 0
        for name, param in mdl.named_parameters():
            temp = param.data.cpu().numpy().reshape(-1)
            param_mat[i, idx:idx + len(temp)] = temp
            idx += len(temp)
    return np.copy(param_mat)

In [17]:
model = models.ResNet18('ResNet')

In [20]:
res1 = get_mdl_params([model])

In [22]:
res1.shape

(1, 11181642)

In [24]:
res2 = SerializationTool.serialize_model(model)

In [26]:
res2.shape

torch.Size([11181642])

In [28]:
res1, res2.as

(array([[ 0.04133107, -0.04457254, -0.04227697, ...,  0.01762635,
          0.01168465,  0.00579628]], dtype=float32),
 tensor([ 0.0413, -0.0446, -0.0423,  ...,  0.0176,  0.0117,  0.0058]))

In [33]:
f"{res2[-1]:.8f}"

'0.00579628'

In [34]:
res2.dtype

torch.float32

In [42]:
res1_l2 = np.sum(res1*res1)
res2_l2 = torch.sum(res2*res2)


In [43]:
res1_l2, res2_l2

(12487.649, tensor(12487.6582))

In [53]:
x = np.array([0,1,2,3,4,5,6,7,8,9,10])
x[[1,2,3,4]]

array([1, 2, 3, 4])

In [55]:
17*35

595

In [56]:
1.74*1024/91

19.57978021978022

In [57]:
x = np.array([1,2,3,4])
split = np.array([5,6,7,8])
x = np.append(x,split)

In [59]:
x

array([1, 2, 3, 4, 5, 6, 7, 8])

In [60]:
np.ndarray(0, dtype=np.int64)

array([], dtype=int64)

In [61]:
num_clients = 3
idx_batch = [np.array([0,1,2]), np.array([3,4,5]), np.array([6,7,8,9])]
{cid: idx_batch[cid] for cid in range(num_clients)}

{0: array([0, 1, 2]), 1: array([3, 4, 5]), 2: array([6, 7, 8, 9])}

In [68]:
np.random.randint(4)

1

In [100]:
data_dir = '../../../data/FCUBE/'
trainset = FCUBE(root=data_dir, train=True, generate=True)

Generate FCUBE data now...
../../../data/FCUBE/fcube_train_X_4000.npy generated.
../../../data/FCUBE/fcube_train_y_4000.npy generated.


In [103]:
trainset.data.shape, trainset.targets.shape

((4000, 3), (4000,))

In [104]:
trainset2 = FCUBE(root=data_dir, train=True, generate=False)

FCUBE data already generated. Load from file ../../../data/FCUBE/fcube_train_X_4000.npy and ../../../data/FCUBE/fcube_train_y_4000.npy...
FCUBE data loaded from local file.


In [105]:
trainset2.data.shape, trainset2.targets.shape 

((4000, 3), (4000,))